**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
!pip install scikit-video
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import random

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(s_0,a_{0},s_{1},a_{1},,...,s_{T-1},a_{T-1},s_{T})=p(s_{0})\prod_{t=1}^{T-1}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The method act in class Agent defines the $\epsilon$-greedy policy, it allows to make a tradeoff between exploration and exploitation by taking an action according to the learned policy with probability $1 - \epsilon$ and  random action with probability $\epsilon$

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

- The array board represents the grid of rewards (cheese and poisonous celles), each time the rat is in some coordinates $(x, y)$ it gets $borad[x, y]$ as a reward and we set the value (reward) of that cell to $0$ since it has been consumed.

- The array position defines the borders of the grid, this way the agent cannot go outside of the grid.


## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        """
        Take a random action from the set of possible actions.
        """
        return np.random.randint(0, self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
      # At each epoch, we restart to a fresh game and get the initial state
      state = env.reset()
      # This assumes that the games will end
      game_over = False

      win = 0
      lose = 0

      while not game_over:
          # The agent performs an action according to the epsilon-greedy policy
          action = agent.act(state)

          # Apply an action to the environment, get the next state, the reward
          # and if the games end
          prev_state = state
          state, reward, game_over = env.act(action)

          # Update the counters
          if reward > 0:
              win = win + reward
          if reward < 0:
              lose = lose - reward
        
      # Save as a mp4
      env.draw(prefix+str(e))

      # Update stats
      score = score + win-lose

      print("Win/lose count {}/{}. Average score ({})"
            .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.5/19.0. Average score (-9.5)
Win/lose count 11.0/13.0. Average score (-5.75)
Win/lose count 9.0/13.0. Average score (-5.166666666666667)
Win/lose count 12.0/12.0. Average score (-3.875)
Win/lose count 11.0/15.0. Average score (-3.9)
Win/lose count 15.0/12.0. Average score (-2.75)
Win/lose count 9.0/17.0. Average score (-3.5)
Win/lose count 9.5/12.0. Average score (-3.375)
Win/lose count 6.0/7.0. Average score (-3.111111111111111)
Win/lose count 11.0/19.0. Average score (-3.6)
Win/lose count 10.0/17.0. Average score (-3.909090909090909)
Win/lose count 10.5/15.0. Average score (-3.9583333333333335)
Win/lose count 6.0/18.0. Average score (-4.576923076923077)
Win/lose count 10.0/14.0. Average score (-4.535714285714286)
Win/lose count 10.0/17.0. Average score (-4.7)
Win/lose count 8.0/9.0. Average score (-4.46875)
Win/lose count 5.0/13.0. Average score (-4.676470588235294)
Win/lose count 9.5/9.0. Average score (-4.388888888888889)
Win/lose count 5.5/16.0. Average score (-4.

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a) = E_{s', a'\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}


Show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}


Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




- We have 

\begin{align*}
Q^{\pi}(s,a) &= E_{p^{\pi}(s_0, a_0, \ldots)}\left(\sum_{t=0}^{\infty} \gamma^t r(s_t, a_t) | s_0 = s, a_0 = a \right) \\
&= E_{p^{\pi}(s_0, a_0, \ldots)}\left(r(s_0, a_0) + \sum_{t=1}^{\infty} \gamma^t r(s_t, a_t) | s_0 = s, a_0 = a \right) \\
&= E_{p^{\pi}(s_0, a_0, \ldots)}\left(r(s_0, a_0) + \gamma\sum_{t=0}^{\infty} \gamma^t r(s_t, a_t) | s_0 = s, a_0 = a \right)  \\
&= E_{p^{\pi}(s, a, s', a' )}\left(E_{p^{\pi}(s_1, a_1, \ldots)}\left(r(s_0, a_0) + \gamma \sum_{t=0}^{\infty}\gamma^t r(s_{t+1}, a_{t+1}) | s_1 = s', a_1 = a' \right) | s_0 = s, a_0 = a \right) \\
&= E_{p^{\pi}(s, a, s', a' )}\left(r(s, a) + \gamma E_{p^{\pi}(s_1, a_1, \ldots)}\left( \sum_{t=0}^{\infty}\gamma^t r(s_{t+1}, a_{t+1}) | s_1 = s', a_1 = a' \right)  \right) \\
&= E_{s' \sim p(.|s, a);a' \sim \pi(.|s')}\left(r(s, a) + \gamma Q^{\pi}(s', a') \right)
\end{align*}

Notice that when we use $p^{\pi}(s, a, s', a')$ or with any subset of random variables, we are in fact using the marginal distribution over the remaining variables.


- We assume that the optimal policy is deterministic or at least of very low entropy, we have then:

\begin{align*}
Q^*(s, a) &= E_{s' \sim p(.|s, a)}\left(r(s, a) + \gamma Q^*(s', \pi^*(s')) \right)\\
&= E_{s'\sim \pi^*(.|s,a)}\left(r(s,a)+\gamma\max_{a'}Q^{*}(s',a') \right)
\end{align*}

Minimizing the loss $\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$ means that we get closer to satisfying the eqation of $Q^*$, in fact it is the equation of the fixed point of the optimal Bellman operator that admits a unique solution. This is what makes minimizing $\mathcal{L}(\theta)$ getting us closer to $Q^*$


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)

    def random_access(self):
        return random.choice(self.memory)

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if (e + 1) % 10 == 0:
            env.draw(prefix+str(e + 1))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return self.model.predict(s.reshape(1, 5, 5, self.n_state)).argmax()

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        on_batch = min(self.batch_size, len(self.memory.memory)) # Number of training samples, it is useful in the beginning when the number of experiences
                                                                 # stored in the replay buffer is inferior to the batch size.
        input_states = np.zeros((on_batch, 5, 5, self.n_state))  # Initialize the input states.
        target_q = np.zeros((on_batch, self.n_action))           # Initialize the target q values.
        
        for i in range(on_batch):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access() # randomy sample an experience from the replay buffer
            input_states[i] = s_
            target_q[i] = self.model.predict(s_.reshape(1, 5, 5, self.n_state)) # Since we train on an experience where input state is related to the action taken.
                                                                                # we only want to backpropagate according to that action and not on all the output actions.
            
            if game_over_:
                ######## FILL IN
                target_q[i, a_] = r_
                
            else:
                ######## FILL IN
                target_q[i, a_] = r_ + self.discount*self.model.predict(n_s_.reshape(1, 5, 5, self.n_state)).max()
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
#         model.add(Dense(50, activation='relu'))
        model.add(Dense(25, activation='relu'))
        model.add(Dense(4, activation='linear'))
        
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model
        

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=1e-3, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train20.mp4'))

Epoch 000/020 | Loss 0.0051 | Win/lose count 1.0/4.0 (-3.0)
Epoch 001/020 | Loss 0.0024 | Win/lose count 5.0/2.0 (3.0)
Epoch 002/020 | Loss 0.0029 | Win/lose count 5.0/5.0 (0.0)
Epoch 003/020 | Loss 0.0022 | Win/lose count 4.5/8.0 (-3.5)
Epoch 004/020 | Loss 0.0114 | Win/lose count 7.0/0 (7.0)
Epoch 005/020 | Loss 0.0032 | Win/lose count 5.0/3.0 (2.0)
Epoch 006/020 | Loss 0.0068 | Win/lose count 9.0/3.0 (6.0)
Epoch 007/020 | Loss 0.0079 | Win/lose count 6.0/4.0 (2.0)
Epoch 008/020 | Loss 0.0070 | Win/lose count 7.5/2.0 (5.5)
Epoch 009/020 | Loss 0.0027 | Win/lose count 3.0/3.0 (0.0)
Epoch 010/020 | Loss 0.0024 | Win/lose count 6.0/2.0 (4.0)
Epoch 011/020 | Loss 0.0020 | Win/lose count 10.5/4.0 (6.5)
Epoch 012/020 | Loss 0.0142 | Win/lose count 8.5/5.0 (3.5)
Epoch 013/020 | Loss 0.0028 | Win/lose count 6.5/7.0 (-0.5)
Epoch 014/020 | Loss 0.0556 | Win/lose count 2.0/2.0 (0.0)
Epoch 015/020 | Loss 0.0078 | Win/lose count 6.5/2.0 (4.5)
Epoch 016/020 | Loss 0.0047 | Win/lose count 4.5/5.0 (

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(8, (3, 3), activation='relu'))
        model.add(Conv2D(16, (2, 2), activation='relu'))
        model.add(Flatten())
        model.add(Dense(4, activation='linear'))
        
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=1e-3, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train20.mp4'))

Epoch 000/020 | Loss 0.0000 | Win/lose count 2.5/5.0 (-2.5)
Epoch 001/020 | Loss 0.0584 | Win/lose count 3.0/2.0 (1.0)
Epoch 002/020 | Loss 0.0092 | Win/lose count 7.5/5.0 (2.5)
Epoch 003/020 | Loss 0.0029 | Win/lose count 4.0/5.0 (-1.0)
Epoch 004/020 | Loss 0.0040 | Win/lose count 11.0/10.0 (1.0)
Epoch 005/020 | Loss 0.0294 | Win/lose count 4.0/3.0 (1.0)
Epoch 006/020 | Loss 0.0031 | Win/lose count 12.5/5.0 (7.5)
Epoch 007/020 | Loss 0.0066 | Win/lose count 3.0/5.0 (-2.0)
Epoch 008/020 | Loss 0.0045 | Win/lose count 8.5/5.0 (3.5)
Epoch 009/020 | Loss 0.0027 | Win/lose count 0.5/1.0 (-0.5)
Epoch 010/020 | Loss 0.0051 | Win/lose count 14.5/6.0 (8.5)
Epoch 011/020 | Loss 0.0023 | Win/lose count 5.0/0 (5.0)
Epoch 012/020 | Loss 0.0518 | Win/lose count 12.0/3.0 (9.0)
Epoch 013/020 | Loss 0.0050 | Win/lose count 10.0/1.0 (9.0)
Epoch 014/020 | Loss 0.0032 | Win/lose count 12.5/3.0 (9.5)
Epoch 015/020 | Loss 0.0061 | Win/lose count 13.0/3.0 (10.0)
Epoch 016/020 | Loss 0.0056 | Win/lose count 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=1e-3, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=1e-3, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 2.5/2.0. Average score (0.5)
Win/lose count 10.0/2.0. Average score (4.25)
Win/lose count 4.5/1.0. Average score (4.0)
Win/lose count 10.0/2.0. Average score (5.0)
Win/lose count 8.0/1.0. Average score (5.4)
Win/lose count 5.5/4.0. Average score (4.75)
Win/lose count 5.5/3.0. Average score (4.428571428571429)
Win/lose count 10.5/3.0. Average score (4.8125)
Win/lose count 9.5/2.0. Average score (5.111111111111111)
Win/lose count 4.0/0. Average score (5.0)
Win/lose count 4.5/0. Average score (4.954545454545454)
Win/lose count 6.0/2.0. Average score (4.875)
Win/lose count 9.0/4.0. Average score (4.884615384615385)
Win/lose count 4.5/3.0. Average score (4.642857142857143)
Win/lose count 4.5/0. Average score (4.633333333333334)
Win/lose count 9.5/2.0. Average score (4.8125)
Win/lose count 4.5/3.0. Average score (4.617647058823529)
Win/lose count 10.5/5.0. Average score (4.666666666666667)
Win/lose count 3.5/1.0. Average score (4.552631578947368)
Win/lose count

In [0]:
HTML(display_videos('cnn_test10.mp4'))

In [0]:
HTML(display_videos('fc_test10.mp4'))

- Both the convolutional and fully connected models perform poorly as the agent tends to get stuck in a portion of the space, this is bad as the agent misses the opportunity of gathering more reward.

- As the temperature decreases, the grid rewards become sparse and the issue becomes more apparent, as the agent rarely encouters reward the training becomes very hard.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix='', eps_final = 0.1, decay_rate = 0.0005):
    """
    This training scheme relies on a decaying epsilon for the epsilon-greedy policy, we start with a high epsilon (great exploration) and
    anneal it down to eps_final (great exploitation).
    """
    # Number of won games
    score = 0
    loss = 0
    eps_start = agent.epsilon
    step = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            eps = eps_final + (eps_start - eps_final)*np.exp(-step*decay_rate)
            step += 1
            agent.set_epsilon(eps)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
            reward -= env.malus_position[env.x, env.y] # In the training process, we penalize the agent for visiting previously visited cells.

            # Update the counters
            if reward > 0:
                win = win + reward
            elif reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if (e + 1) % 10 == 0:
            env.draw(prefix+str(e + 1))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.1f}/{:.1f} ({}) | explore_proba {:.2f}"
              .format(e, epoch, loss, win, lose, win-lose, eps))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale = 16

        self.to_draw = np.zeros((max_time + 2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        self.malus_position[self.x, self.y] = 0.1 # each previously visited cell holds a negative reward now.
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                        self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2) # The information about previously visited states is now incorporated into the state representation.
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus
        
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        self.malus_position[self.x, self.y] = 0.1
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [0]:
# Training
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=20 # set small when debugging

env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=1e-3, epsilon = 0.9, memory_size=2000, batch_size = 32, n_state=3)
train_explore(agent, env, epochs_train, prefix='fc_train_explore')
HTML(display_videos('fc_train_explore100.mp4'))

Epoch 000/100 | Loss 0.0209 | Win/lose count 10.0/36.6 (-26.600000000000126) | explore_proba 0.82
Epoch 001/100 | Loss 0.0250 | Win/lose count 8.0/31.1 (-23.100000000000144) | explore_proba 0.75
Epoch 002/100 | Loss 0.0151 | Win/lose count 6.8/33.4 (-26.600000000000126) | explore_proba 0.69
Epoch 003/100 | Loss 0.0087 | Win/lose count 11.6/22.2 (-10.600000000000055) | explore_proba 0.64
Epoch 004/100 | Loss 0.0089 | Win/lose count 7.6/30.2 (-22.600000000000122) | explore_proba 0.58
Epoch 005/100 | Loss 0.0093 | Win/lose count 5.6/30.7 (-25.10000000000013) | explore_proba 0.54
Epoch 006/100 | Loss 0.0048 | Win/lose count 7.6/28.2 (-20.6000000000001) | explore_proba 0.50
Epoch 007/100 | Loss 0.0088 | Win/lose count 12.8/20.9 (-8.100000000000035) | explore_proba 0.46
Epoch 008/100 | Loss 0.0197 | Win/lose count 12.0/27.1 (-15.100000000000039) | explore_proba 0.42
Epoch 009/100 | Loss 0.0062 | Win/lose count 14.0/24.6 (-10.600000000000026) | explore_proba 0.39
Epoch 010/100 | Loss 0.0416 |

In [0]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_cnn = DQN_CNN(size, lr=1e-3, epsilon = 0.9, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent_cnn, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore100.mp4'))

Epoch 000/100 | Loss 0.0140 | Win/lose count 12.4/32.0 (-19.6000000000001) | explore_proba 0.82
Epoch 001/100 | Loss 0.0050 | Win/lose count 9.6/25.7 (-16.100000000000072) | explore_proba 0.75
Epoch 002/100 | Loss 0.0077 | Win/lose count 9.6/32.7 (-23.100000000000115) | explore_proba 0.69
Epoch 003/100 | Loss 0.0073 | Win/lose count 9.2/25.8 (-16.600000000000062) | explore_proba 0.64
Epoch 004/100 | Loss 0.0105 | Win/lose count 6.8/34.4 (-27.600000000000126) | explore_proba 0.58
Epoch 005/100 | Loss 0.0396 | Win/lose count 6.4/27.5 (-21.100000000000108) | explore_proba 0.54
Epoch 006/100 | Loss 0.0308 | Win/lose count 6.0/29.6 (-23.600000000000104) | explore_proba 0.50
Epoch 007/100 | Loss 0.0092 | Win/lose count 11.6/33.2 (-21.60000000000011) | explore_proba 0.46
Epoch 008/100 | Loss 0.0061 | Win/lose count 17.2/28.8 (-11.600000000000055) | explore_proba 0.42
Epoch 009/100 | Loss 0.0066 | Win/lose count 11.2/24.3 (-13.100000000000053) | explore_proba 0.39
Epoch 010/100 | Loss 0.0086 |

In [0]:
# Evaluation
test(agent,env,epochs_test,prefix='fc_test_explore')
HTML(display_videos('fc_test_explore19.mp4'))

Win/lose count 24.5/5.0. Average score (19.5)
Win/lose count 14.0/2.0. Average score (15.75)
Win/lose count 19.0/2.0. Average score (16.166666666666668)
Win/lose count 20.5/4.0. Average score (16.25)
Win/lose count 16.0/3.0. Average score (15.6)
Win/lose count 18.5/1.0. Average score (15.916666666666666)
Win/lose count 15.5/4.0. Average score (15.285714285714286)
Win/lose count 20.5/3.0. Average score (15.5625)
Win/lose count 23.5/2.0. Average score (16.22222222222222)
Win/lose count 11.0/3.0. Average score (15.4)
Win/lose count 20.5/3.0. Average score (15.590909090909092)
Win/lose count 19.5/7.0. Average score (15.333333333333334)
Win/lose count 10.5/3.0. Average score (14.73076923076923)
Win/lose count 18.0/0. Average score (14.964285714285714)
Win/lose count 14.0/6.0. Average score (14.5)
Win/lose count 25.0/5.0. Average score (14.84375)
Win/lose count 21.0/5.0. Average score (14.911764705882353)
Win/lose count 17.5/1.0. Average score (15.0)
Win/lose count 22.5/7.0. Average score (1

In [0]:
# Evaluation
test(agent_cnn,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore19.mp4'))

Win/lose count 25.0/4.0. Average score (21.0)
Win/lose count 22.5/4.0. Average score (19.75)
Win/lose count 18.0/6.0. Average score (17.166666666666668)
Win/lose count 23.0/4.0. Average score (17.625)
Win/lose count 17.0/1.0. Average score (17.3)
Win/lose count 21.0/1.0. Average score (17.75)
Win/lose count 20.0/2.0. Average score (17.785714285714285)
Win/lose count 11.5/0. Average score (17.0)
Win/lose count 21.5/2.0. Average score (17.27777777777778)
Win/lose count 25.5/2.0. Average score (17.9)
Win/lose count 22.5/6.0. Average score (17.772727272727273)
Win/lose count 21.0/1.0. Average score (17.958333333333332)
Win/lose count 22.5/6.0. Average score (17.846153846153847)
Win/lose count 18.5/3.0. Average score (17.678571428571427)
Win/lose count 26.5/5.0. Average score (17.933333333333334)
Win/lose count 21.5/1.0. Average score (18.09375)
Win/lose count 17.0/3.0. Average score (17.852941176470587)
Win/lose count 13.0/1.0. Average score (17.52777777777778)
Win/lose count 21.5/4.0. Ave

In [0]:
HTML(display_videos('cnn_test_explore6.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***